In [1]:
import pandas as pd
from utils import Timer
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
file = "/mnt/DP_disk1/ht/datasets/autofe/fraud_detect/card_transaction.v1.csv"
with Timer("read train data"):
    # Reading Data
    df =  pd.read_csv(file)
df

read train data took 23.536249216645956 sec


,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020,2,27,22:23,$-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386896,1999,1,2020,2,27,22:24,$54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386897,1999,1,2020,2,28,07:43,$59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No
24386898,1999,1,2020,2,28,20:10,$43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No


In [3]:
train_df = df[df['Year'] < 2018].reset_index(drop=True)
valid_df = df[df['Year'] == 2018].reset_index(drop=True)
test_df = df[df['Year'] > 2018].reset_index(drop=True)

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.core.metrics import make_scorer
from sklearn.metrics import average_precision_score, precision_recall_curve, auc

def score_func(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    result = auc(recall, precision)
    return result
    
ag_aucpr_scorer = make_scorer(name='aucpr',
                                 score_func=score_func,
                                 optimum=1,
                                 greater_is_better=True)

model = TabularPredictor(label="Is Fraud?", eval_metric=ag_aucpr_scorer)
predictor = model.fit(train_data=train_df, tuning_data=valid_df, hyperparameters={
    "GBM": {},
    "RF": {},
    "XT": {},
    "KNN": {},
    "LR": {},
})

No path specified. Models will be saved in: "AutogluonModels/ag-20230808_061301/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230808_061301/"
AutoGluon Version:  0.7.0
Python Version:     3.8.0
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023
Train Data Rows:    20604847
Train Data Columns: 14
Tuning Data Rows:    1721615
Tuning Data Columns: 14
Label Column: Is Fraud?
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['No', 'Yes']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type

In [5]:
predictor.evaluate(test_df)

Evaluation: aucpr on test data: 0.5718009250361189
Evaluations on test data:
{
    "aucpr": 0.5718009250361189,
    "accuracy": 0.9991890073858083,
    "balanced_accuracy": 0.6082806657730643,
    "mcc": 0.4476757856131331,
    "roc_auc": 0.9995129064540973,
    "f1": 0.3510679611650485,
    "precision": 0.9262295081967213,
    "recall": 0.21657882127455677
}


{'aucpr': 0.5718009250361189,
 'accuracy': 0.9991890073858083,
 'balanced_accuracy': 0.6082806657730643,
 'mcc': 0.4476757856131331,
 'roc_auc': 0.9995129064540973,
 'f1': 0.3510679611650485,
 'precision': 0.9262295081967213,
 'recall': 0.21657882127455677}